# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_original = "output_data\weater_data.csv"
weather_data = pd.read_csv(weather_original)
weather_df = pd.DataFrame(weather_data)
weather_df.head()

,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Date,Max. Temperature (F),Cloudiness (%),Humidity (%),Wind Speed
0,0,2206939,Bluff,NZ,-46.6000,168.3333,1592697600,44.11,15,79,7.76
1,1,2419992,Kankan,GN,10.3854,-9.3057,1592697600,76.86,36,78,4.81
2,2,3414079,Olafsvik,IS,64.8945,-23.7142,1592697600,51.17,72,87,2.37
3,3,1265996,Kothāpet,IN,19.3333,79.4833,1592697600,82.63,100,70,4.36
4,4,4030556,Rikitea,PF,-23.1203,-134.9692,1592697600,76.06,62,83,11.81


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [8]:
# Use Lat and Lng as locations and Humidity as the weight
locations = weather_df[['Latitude', 'Longitude']].astype(float)
hum_weight = weather_df['Humidity (%)'].astype(float)

In [9]:
# Add Heatmap layer to map
fig = gmaps.figure(center=(48.0, 0.0), zoom_level = 2)

heat_layer1 = gmaps.heatmap_layer(locations, weights = hum_weight, 
                                 dissipating = False, max_intensity = 100,
                                 point_radius = 1, opacity=1)

fig.add_layer(heat_layer1)

plt.savefig("hum_lat_heatmap.png")
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
# Narrow down cities to fit specific weather conditions per directions of:
# A max temperature lower than 80 degrees but higher than 70.
condition1 = weather_df.loc[weather_df["Max. Temperature (F)"]<80,:]
condition2 = condition1.loc[weather_df["Max. Temperature (F)"]>70,:]

In [18]:
# Wind speed less than 10 mph.
condition3 = condition2.loc[condition2["Wind Speed"]<10,:]

In [19]:
# Zero cloudiness.
conditions_final = condition3.loc[condition3["Cloudiness (%)"]==0,:]

In [23]:
# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Removed non-wanted conditions by locating only those that are wanted and replacing original data if does not fit wanted.
conditions_final

,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Date,Max. Temperature (F),Cloudiness (%),Humidity (%),Wind Speed
93,93,3445026,Vila Velha,BR,-20.3297,-40.2925,1592697600,74.21,0,80,9.89
96,96,2033574,Zhangjiakou Shi Xuanhua Qu,CN,40.6103,115.0447,1592697600,77.31,0,58,4.16
118,118,2510693,Talavera de la Reina,ES,39.9635,-4.8308,1592697600,70.36,0,37,1.59
206,206,323828,Anamur,TR,36.0751,32.8369,1592697600,76.95,0,54,4.85
220,220,2210247,Tripoli,LY,32.8752,13.1875,1592697600,72.07,0,70,9.69
227,227,209598,Manono,CD,-7.3000,27.4167,1592697600,70.25,0,50,5.30
249,249,124291,Mehrān,IR,33.1222,46.1646,1592697600,79.32,0,23,6.82
310,310,2517512,Fuente Palmera,ES,37.7049,-5.0996,1592697600,70.95,0,45,4.83
336,336,371745,Kutum,SD,14.2000,24.6667,1592697600,73.29,0,20,4.38
340,340,2541479,Nador,MA,35.1740,-2.9287,1592697600,73.35,0,70,1.25


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# Store into variable named hotel_df
hotel_df = pd.DataFrame()

In [25]:
# Add a "Hotel Name" column to the DataFrame
hotel_wanted = pd.DataFrame(conditions_final)
hotel_wanted['Hotel Name'] = ""
hotel_wanted.head()

,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Date,Max. Temperature (F),Cloudiness (%),Humidity (%),Wind Speed,Hotel Name
93,93,3445026,Vila Velha,BR,-20.3297,-40.2925,1592697600,74.21,0,80,9.89,
96,96,2033574,Zhangjiakou Shi Xuanhua Qu,CN,40.6103,115.0447,1592697600,77.31,0,58,4.16,
118,118,2510693,Talavera de la Reina,ES,39.9635,-4.8308,1592697600,70.36,0,37,1.59,
206,206,323828,Anamur,TR,36.0751,32.8369,1592697600,76.95,0,54,4.85,
220,220,2210247,Tripoli,LY,32.8752,13.1875,1592697600,72.07,0,70,9.69,


In [36]:
# Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "types": "lodging",
    "keyword": "Hotel",
    "key": g_key
}

In [46]:
# Hit the Google Places API for each city's coordinates.
for index, row in hotel_wanted.iterrows():

    # iterate through:
    latitude = row['Latitude']
    longitude = row['Longitude']
    city = row['City']
    country = row['Country']

    # Create URL and call to API
    print(f"Pulling results for Index # {index}, the city of {city}.")
    hotel_response = requests.get(f"https://maps.googleapis.com/maps/api/place/textsearch/json?location={latitude},{longitude}&radius=5000&type=hotel&key={g_key}").json()
    
    # Print Iteration Results
    hotel_results = hotel_response['results']
    
    try:
        hotel_name = hotel_response['results'][7]['name']
        print(f"Results show the closest hotel to {city} is at {latitude} , {longitude} and is named {hotel_name}.")
        
        hotel_wanted.loc[index, "Hotel Name"] = hotel_name
        
    except (KeyError, IndexError):
        print("N/A")
        
    print("~~~~~~~~~~")

Pulling results for Index # 93, the city of Vila Velha.
Results show the closest hotel to Vila Velha is at -20.3297 , -40.2925 and is named Golden Tulip Porto Vitória.
~~~~~~~~~~
Pulling results for Index # 96, the city of Zhangjiakou Shi Xuanhua Qu.
Results show the closest hotel to Zhangjiakou Shi Xuanhua Qu is at 40.6103 , 115.0447 and is named 张家口市宣化区政府.
~~~~~~~~~~
Pulling results for Index # 118, the city of Talavera de la Reina.
Results show the closest hotel to Talavera de la Reina is at 39.9635 , -4.8308 and is named BBVA.
~~~~~~~~~~
Pulling results for Index # 206, the city of Anamur.
Results show the closest hotel to Anamur is at 36.0751 , 32.8369 and is named Nokta Cafe Bar.
~~~~~~~~~~
Pulling results for Index # 220, the city of Tripoli.
Results show the closest hotel to Tripoli is at 32.8752 , 13.1875 and is named Hotel Al-Safwa.
~~~~~~~~~~
Pulling results for Index # 227, the city of Manono.
Results show the closest hotel to Manono is at -7.3 , 27.4167 and is named Territ

In [51]:
# Store the first Hotel result into the DataFrame
# Shown in iteration above: hotel_wanted.loc[index, "Hotel Name"] = hotel_name
hotel_wanted.head()

,Unnamed: 0,City ID,City,Country,Latitude,Longitude,Date,Max. Temperature (F),Cloudiness (%),Humidity (%),Wind Speed,Hotel Name
93,93,3445026,Vila Velha,BR,-20.3297,-40.2925,1592697600,74.21,0,80,9.89,Golden Tulip Porto Vitória
96,96,2033574,Zhangjiakou Shi Xuanhua Qu,CN,40.6103,115.0447,1592697600,77.31,0,58,4.16,张家口市宣化区政府
118,118,2510693,Talavera de la Reina,ES,39.9635,-4.8308,1592697600,70.36,0,37,1.59,BBVA
206,206,323828,Anamur,TR,36.0751,32.8369,1592697600,76.95,0,54,4.85,Nokta Cafe Bar
220,220,2210247,Tripoli,LY,32.8752,13.1875,1592697600,72.07,0,70,9.69,Hotel Al-Safwa


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_wanted.iterrows()]
locations = hotel_wanted[["Latitude", "Longitude"]]

In [61]:
# Add marker layer ontop of heat map
marker_layer1 = gmaps.marker_layer(locations, hover_text='', label='', info_box_content=hotel_info)

# Display figure
fig.add_layer(heat_layer1)
fig.add_layer(marker_layer1)
plt.savefig("hotel_marker.png")

fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>